In [1]:
########
# LC generator for RRLyrs
########

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
bg_color = "#000000"
use_period_color = False
R_color = 1
G_color = 1
B_color = 1
var_Type = "rrlyr"
pltSIZE = 128
pltDPI = 72
stackSize = 25000
generateClones = True
generatePeriodTXT = True
generateOnlyTXTData = True

webaddress_base = "http://www.astrouw.edu.pl/ogle/ogle3/OIII-CVS/lmc/" + var_Type + "/"
web_data_folder = "phot/I/OGLE-LMC-" + var_Type.upper() + "-"

ident = pd.read_csv(webaddress_base + "ident.dat", header=None, delim_whitespace=True, usecols=[0,3])
ident.columns = ['id','type']

datacount = ident.shape[0]
cloneGenRatio = int((stackSize-datacount)/datacount)
cloneNumber = stackSize-datacount

rrab_data = pd.read_csv(webaddress_base + "RRab.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
rrc_data = pd.read_csv(webaddress_base + "RRc.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
rre_data = pd.read_csv(webaddress_base + "RRe.dat", header=None, delim_whitespace=True, usecols=[0,3,5])
rrd_data = pd.read_csv(webaddress_base + "RRd.dat", header=None, delim_whitespace=True, usecols=[0,11,13])
rrd_data.columns = [0,3,5]

types = [rrab_data,rrc_data,rre_data,rrd_data]
rrlyr_data = pd.concat(types,ignore_index=True)

rrlyr_data.reset_index(drop=True)
rrlyr_data.columns = ['id','period','T_max']
rrlyr_data['id'] = pd.Series(rrlyr_data['id']).str[15:20]
rrlyr_data.set_index('id', inplace=True)

#rrlyr_data['id'] = pd.to_numeric(rrlyr_data['id'])
#rrlyr_data = rrlyr_data.to_numpy(copy=True)

In [12]:
print("Available data:", datacount)
print("Desired number of data:", stackSize)
print("Will generate", cloneNumber , "clones")

Available data: 24906
Desired number of data: 25000
Will generate 94 clones


In [4]:
!pwd

/Users/tomi/Astronomy/MTA_CSFK/ML_OGLE/OGLE3/scripts/LC_Gen


In [16]:
webaddress = webaddress_base + web_data_folder
targetFolder = "../../non_colored_txt/"

if generateClones:
    cloneCounter = 0
parentCounter = 0

for rrlrow in range(datacount):   
#    if ident['type'][rrlrow] != "RRd":
#        continue
#    else:
    star_identifier = str(f'{rrlrow+1:05d}')
    data_filename = webaddress + star_identifier + ".dat"
    
    filename = targetFolder + var_Type + "_" + star_identifier

    data = pd.read_csv(data_filename, delim_whitespace=True, header=None, usecols=[0,1,2])
    data.columns = ["date", "mag","error"]
                
    T_max = rrlyr_data.loc[star_identifier].T_max
    period = rrlyr_data.loc[star_identifier].period
        
    if use_period_color:
#         print("Color coded period")
        if 0 < period and period <=1:
            R_color = period
            G_color = 0
            B_color = 0
        if 1 < period and period <=10:
            R_color = 1
            G_color = period/10
            B_color = 0
        if 10 < period and period <=100:
            R_color = 1
            G_color = 1
            B_color = period/100
        if period > 100:
            R_color = 1
            G_color = 1
            B_color = 1
    else:
#         print("Not using color coded period")
        print("Color: R:", R_color, "G:", G_color, "B:", B_color)

    
#     print("Color: R:", R_color, "G:", G_color, "B:", B_color)
#     print(T_max, "--", period)
        
    flux_cont = []
    flux_norm_cont = []
    phase_cont = []
    
    mag_cont = []
    t_cont = []
    error_cont = []
    
    for i in data.index:
        mag = data['mag'][i]
        t = data['date'][i]
        error = data['error'][i]
        mag_cont.append(mag)
        t_cont.append(t)
        error_cont.append(error)
        
        phase = (t-T_max)/period - int((t-T_max)/period)
        flux = 10**(-0.4*(mag-25))
        flux_cont.append(flux)
        phase_cont.append(phase)
        #print(mag , "--",  flux)
        
    if generateClones and cloneCounter != cloneNumber:
#         if rrlrow == 0:
#             cloneGenRatio = round((stackSize-datacount)/datacount)
#             floatRatio = (stackSize-datacount)/datacount
#         else:
#             # Checking and adjusting clone number
#             remParent = datacount-parentCounter
#             remAll = (stackSize-cloneCounter-parentCounter)-(datacount-parentCounter)
#             cloneGenRatio = round(remAll/remParent)
#             floatRatio = remAll/remParent
        cloneGenRatio = 1
        
        print("===================")
        max_error = max(error_cont)
        print("Max error:", max_error)
        error_cont = np.random.normal(0, max_error/3, len(error_cont))
        print("Error array generated for " + var_Type + "_" + star_identifier)
         
        for cl in range(cloneGenRatio):
            if cloneCounter == cloneNumber:
                continue
            
            filename_clone = filename + "_clone_" + str(cl)
            
            flux_clone_cont = []
            flux_clone_norm_cont = []
            rand_error_array = np.random.choice(error_cont,len(error_cont))
            mag_clone_cont = mag_cont + rand_error_array
            
            print("Clone generation", cl)
            
            if generatePeriodTXT:
                periodfile = open(filename_clone + ".txt","w+")
                periodfile.write(str(period))
                periodfile.close()
                print("Period data text file generated for" + filename_clone)
                if generateOnlyTXTData:
                    cloneCounter += 1
                    continue
                    
            for c in range(len(mag_clone_cont)):
                flux_clone = 10**(-0.4*(mag_clone_cont[c]-25))
                flux_clone_cont.append(flux_clone)
            
            max_clone_flux = max(flux_clone_cont)
            for i in flux_clone_cont:
                flux_clone_norm = (i/max_clone_flux)
                flux_clone_norm_cont.append(flux_clone_norm)

            phase_cont = np.array(phase_cont)
            flux_clone_norm_cont = np.array(flux_clone_norm_cont)
            
            ###     
            # Plotting the folded lightcurve for cloned variable
            ### 
            plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

            plt.plot(phase_cont, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
            plt.plot(phase_cont+1, flux_clone_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

            filename_clone = filename_clone +".png"

            print("Saving " + filename)
            plt.tight_layout(0)
#             plt.gcf().savefig(filename_clone, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#             plt.show()
            plt.close()
            
            cloneCounter += 1
        
            ###     
            # End of plotting for the clone(s)
            ### 

            # 
# Creating data for the original variable star
# 

    if generatePeriodTXT:
        periodfile = open(filename + ".txt","w+")
        periodfile.write(str(period))
        periodfile.close()
        print("Period data text file generated for" + filename)
        if generateOnlyTXTData:
            parentCounter += 1
            continue
            
    max_flux = max(flux_cont)

    for i in flux_cont:
        flux_norm = (i/max_flux)
        flux_norm_cont.append(flux_norm)

#if len(phase_cont) == len(flux_norm_cont):
    phase_cont = np.array(phase_cont)
    flux_norm_cont = np.array(flux_norm_cont)

    plt.figure(num=None, figsize=(pltSIZE/pltDPI, pltSIZE/pltDPI), dpi=pltDPI, facecolor=bg_color)

    plt.plot(phase_cont, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)
    plt.plot(phase_cont+1, flux_norm_cont, color=(R_color,G_color,B_color), marker='.', linestyle='none', markersize=0.7)

    plt.axis('off')

    filename = filename + ".png"
    print("Saving ORIGINAL", filename)
    plt.tight_layout(0)
#     plt.gcf().savefig(filename, bbox_inches=None, pad_inches=0, facecolor=bg_color, edgecolor='none', dpi='figure')
#     plt.show()
    plt.close()

    parentCounter += 1


Color: R: 1 G: 1 B: 1
Max error: 0.109
Error array generated for rrlyr_00001
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00001_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00001
Color: R: 1 G: 1 B: 1
Max error: 0.081
Error array generated for rrlyr_00002
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00002_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00002
Color: R: 1 G: 1 B: 1
Max error: 0.087
Error array generated for rrlyr_00003
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00003_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00003
Color: R: 1 G: 1 B: 1
Max error: 0.094
Error array generated for rrlyr_00004
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00004_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00004
Color: R: 1 G: 1 B: 1
Max error:

Color: R: 1 G: 1 B: 1
Max error: 0.161
Error array generated for rrlyr_00033
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00033_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00033
Color: R: 1 G: 1 B: 1
Max error: 0.142
Error array generated for rrlyr_00034
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00034_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00034
Color: R: 1 G: 1 B: 1
Max error: 0.129
Error array generated for rrlyr_00035
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00035_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00035
Color: R: 1 G: 1 B: 1
Max error: 0.11699999999999999
Error array generated for rrlyr_00036
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00036_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00036
Color: R: 1 G: 1 B

Color: R: 1 G: 1 B: 1
Max error: 0.177
Error array generated for rrlyr_00065
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00065_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00065
Color: R: 1 G: 1 B: 1
Max error: 0.099
Error array generated for rrlyr_00066
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00066_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00066
Color: R: 1 G: 1 B: 1
Max error: 0.11900000000000001
Error array generated for rrlyr_00067
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00067_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00067
Color: R: 1 G: 1 B: 1
Max error: 0.19899999999999998
Error array generated for rrlyr_00068
Clone generation 0
Period data text file generated for../../non_colored_txt/rrlyr_00068_clone_0
Period data text file generated for../../non_colored_txt/rrlyr_00068
Colo

Period data text file generated for../../non_colored_txt/rrlyr_00097
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00098
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00099
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00100
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00101
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00102
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00103
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00104
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00105
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00106
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00107
Color: R: 1 G: 1 B: 1

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00189
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00190
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00191
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00192
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00193
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00194
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00195
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00196
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00197
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00198
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_00199

...

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24522
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24523
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24524
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24525
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24526
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24527
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24528
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24529
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24530
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24531
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24532

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24613
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24614
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24615
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24616
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24617
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24618
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24619
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24620
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24621
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24622
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24623

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24704
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24705
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24706
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24707
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24708
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24709
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24710
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24711
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24712
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24713
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24714

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24795
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24796
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24797
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24798
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24799
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24800
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24801
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24802
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24803
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24804
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24805

Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24887
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24888
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24889
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24890
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24891
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24892
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24893
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24894
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24895
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24896
Color: R: 1 G: 1 B: 1
Period data text file generated for../../non_colored_txt/rrlyr_24897